In [2]:
%run common_imports.py
import spyglass.position.v1 as sgp
from spyglass.position import PositionOutput

dj.config['display.limit'] = 10**3
%matplotlib qt
plt.style.use(["seaborn-v0_8-talk"])
%load_ext autoreload
%autoreload 2
pd.options.display.max_rows = 1000

[2025-12-23 19:12:00,496][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-23 19:12:00,512][INFO]: Connected anirudh@172.16.102.154:3306


### Insert parameters

In [11]:
bodypart1 = "forepawR"
bodypart2 = "forepawL"
bodypart3 = "hindpawR"
bodypart4 = "hindpawL"

centroid_params_name = "four_paw_centroid"
# centroid_params = {
#     "centroid_method": "four_led_centroid",
#     "points": {
#         "greenLED": bodypart1,
#         "redLED_L": bodypart2,
#         "redLED_C": bodypart3,
#         "redLED_R": bodypart4,
#     },
#     "interpolate": True,
#     "interp_params": {"max_cm_to_interp": 15},
#     "smooth": True,
#     "smoothing_params": {"smoothing_duration": 0.05, "smooth_method": "moving_avg"},
#     "max_LED_separation": 30,
#     "speed_smoothing_std_dev": 0.100,
# }

centroid_params = {
    "centroid_method": "four_paw_centroid",
    "points": {
        "paw_FL": bodypart2,  # dict keys are the required keys in _key_to_points
        "paw_FR": bodypart1,
        "paw_HL": bodypart4,
        "paw_HR": bodypart3,
    },
    "interpolate": True,
    "interp_params": {"max_cm_to_interp": 15},
    "smooth": True,
    "smoothing_params": {"smoothing_duration": 0.05, "smooth_method": "moving_avg"},
    "max_LED_separation": 30, 
    "speed_smoothing_std_dev": 0.1,
}


sgp.DLCCentroidParams.insert1(
    {"dlc_centroid_params_name": centroid_params_name, "params": centroid_params},
    skip_duplicates=True,
)

# sgp.DLCOrientationParams.insert1(
#     {
#         "dlc_orientation_params_name": "four_paw_centoird",
#         "params": {
#             "orient_method": "red_led_bisector",
#             "led1": bodypart2,  
#             "led2": bodypart1,  
#             "led3": bodypart3,
#             "orientation_smoothing_std_dev": 0.001,
#         },
#     },
#     skip_duplicates=True,
#  )


paw_orient_params = {
    "orient_method": "four_paw_body_orientation",
    "front_left":  bodypart2,
    "front_right": bodypart1,
    "hind_left":   bodypart4,
    "hind_right":  bodypart3,
    "orientation_smoothing_std_dev": 0.001, # keep or tune
}

sgp.DLCOrientationParams.insert_params("paws_default", paw_orient_params, skip_duplicates=True)




### Begin loop

In [12]:
sgc.CameraDevice()

camera_name,meters_per_pixel height / width of pixel in meters,manufacturer,model,lens,camera_id
Camera 1,0.0006917,Allied Vision,AVT Manta,Theia,0
senor run camera,0.00172,Allied Vision,Manta,unknown,1
sleep box camera,0.00211,Allied Vision,Manta,unknown,0
SleepBox_camera,0.000842,Manta,unknown,unknown,0
w-track camera,0.00211,Allied Vision,Manta,unknown2,1


In [13]:
sgc.VideoFile() & session_restrict

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),video_file_num,camera_name,video_file_object_id the object id of the file object
wilbur20210512_.nwb,1,0,sleep box camera,10ccb3e3-a5e6-487e-afc1-336f004ba34c
wilbur20210512_.nwb,2,1,w-track camera,3c4b28bd-49ae-4479-bfe2-e12986854b63
wilbur20210512_.nwb,3,0,sleep box camera,ae55822c-f2f4-4a5e-8a40-7b1fcab1f26a
wilbur20210512_.nwb,4,1,w-track camera,cd538c7b-88bf-4194-b3a4-cb15ae119a72
wilbur20210512_.nwb,5,0,sleep box camera,1f0d9adb-a2b3-431c-baf8-82d6867b2bb8
wilbur20210512_.nwb,6,1,w-track camera,1146844d-c2e6-47b0-9494-79b6bcac218b
wilbur20210512_.nwb,7,0,sleep box camera,0de5dfe9-c504-466a-ac9f-8ebe35721bdc
wilbur20210512_.nwb,8,1,w-track camera,6b8edaec-edf3-48e5-8700-632a86bc2fa4


In [14]:
camera_name = "w-track camera"
matching_rows = (sgc.VideoFile & session_restrict) & {"camera_name": camera_name, "video_file_num": 1} & "epoch !=2"
matching_rows

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),video_file_num,camera_name,video_file_object_id the object id of the file object
wilbur20210512_.nwb,4,1,w-track camera,cd538c7b-88bf-4194-b3a4-cb15ae119a72
wilbur20210512_.nwb,6,1,w-track camera,1146844d-c2e6-47b0-9494-79b6bcac218b
wilbur20210512_.nwb,8,1,w-track camera,6b8edaec-edf3-48e5-8700-632a86bc2fa4


In [15]:
sgp.DLCModelSelection()

project_name name of DLC project,dlc_model_name User-friendly model name,dlc_model_params_name
Senor-WtrackSep5-trainset95shuffle1,WtrackSep5-trainset95shuffle1model,WtrackSep5
WtrackSep5-trainset95shuffle1,WtrackSep5-trainset95shuffle1model,WtrackSep5


In [16]:
model_key = (sgp.DLCModelSelection() & {"project_name": "WtrackSep5-trainset95shuffle1"}).fetch1("KEY") 
bodyparts = [f'{bodypart1}', f'{bodypart2}', f'{bodypart3}', f'{bodypart4}']

In [17]:
smooth_interp_params = {'dlc_si_params_name': 'just_nan_gap_5cm',
 'params': {'smooth': False,
  'interpolate': False,
  'likelihood_thresh': 0.95,
  'max_cm_between_pts': 5,
  'num_inds_to_span': 20}}

sgp.DLCSmoothInterpParams.insert1(smooth_interp_params, skip_duplicates = True)
sgp.DLCSmoothInterpParams()

dlc_si_params_name name for this set of parameters,params dictionary of parameters
default,=BLOB=
default_interp_correction,=BLOB=
default_interp_workaround,=BLOB=
just_nan,=BLOB=
just_nan_gap_2cm,=BLOB=
just_nan_gap_5cm,=BLOB=


In [18]:
matching_rows

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),video_file_num,camera_name,video_file_object_id the object id of the file object
wilbur20210512_.nwb,4,1,w-track camera,cd538c7b-88bf-4194-b3a4-cb15ae119a72
wilbur20210512_.nwb,6,1,w-track camera,1146844d-c2e6-47b0-9494-79b6bcac218b
wilbur20210512_.nwb,8,1,w-track camera,6b8edaec-edf3-48e5-8700-632a86bc2fa4


In [29]:
for row in matching_rows:
    col1val = row["nwb_file_name"]
    if "wilbur20210512" in col1val:  # *** change depending on rat/day!!!
        col2val = row["epoch"]
        col3val = row["video_file_num"]

        ##insert pose estimation task
        key = {"nwb_file_name": nwb_copy_file_name,
               "epoch": col2val,
               "video_file_num": col3val,
               **model_key,
               }
        
        dlc_pose_estimation = sgp.DLCPoseEstimationSelection()  # Create an instance

        pose_estimation_key = dlc_pose_estimation.insert_estimation_task(
            key,
            task_mode="trigger", #trigger or load
            params={"gputouse": 0, "videotype": "mp4"},
            )

        ##populate DLC Pose Estimation
        # sgp.DLCPoseEstimation().populate(pose_estimation_key)

        #start smooth interpolation
        si_params_name = "just_nan_gap_5cm"
        si_key = pose_estimation_key.copy()
        fields = list(sgp.DLCSmoothInterpSelection.fetch().dtype.fields.keys())
        si_key = {key: val for key, val in si_key.items() if key in fields}
        # bodyparts = 
        sgp.DLCSmoothInterpSelection.insert(
            [
                {
                    **si_key,
                    "bodypart": bodypart,
                    "dlc_si_params_name": si_params_name,
                }
                for bodypart in bodyparts
            ],
            skip_duplicates=True,
        )
        
        #-----------------
        # sgp.DLCSmoothInterp().populate({**si_key, "dlc_si_params_name": "just_nan_gap_5cm"})
        #-----------------
        
        
        (
            sgp.DLCSmoothInterp() & {**si_key, "bodypart": bodyparts[0]}
        ).fetch1_dataframe().plot.scatter(x="x", y="y", s=1, figsize=(5, 5))

        #smoothinterpcohort
        cohort_key = si_key.copy()
        if "bodypart" in cohort_key:
            del cohort_key["bodypart"]
        if "dlc_si_params_name" in cohort_key:
            del cohort_key["dlc_si_params_name"]
        cohort_key["dlc_si_cohort_selection_name"] = centroid_params_name
        cohort_key["bodyparts_params_dict"] = {
            f"{bodypart1}": si_params_name,
            f"{bodypart2}": si_params_name,
            f"{bodypart3}": si_params_name,
            f"{bodypart4}": si_params_name
        }
        sgp.DLCSmoothInterpCohortSelection().insert1(
            cohort_key, skip_duplicates=True
        )
        sgp.DLCSmoothInterpCohort.populate(cohort_key)

        ##DLC Centroid
        centroid_params_name = centroid_params_name
        centroid_key = cohort_key.copy()
        fields = list(sgp.DLCCentroidSelection.fetch().dtype.fields.keys())
        centroid_key = {
            key: val for key, val in centroid_key.items() if key in fields
        }
        centroid_key["dlc_centroid_params_name"] = centroid_params_name
        sgp.DLCCentroidSelection.insert1(centroid_key, skip_duplicates=True)
        print(centroid_key)
        # #-----------------
        sgp.DLCCentroid.populate(centroid_key)
        # #-----------------
        
        (sgp.DLCCentroid() & centroid_key).fetch1_dataframe().plot.scatter(
            x="position_x",
            y="position_y",
            c="speed",
            colormap="viridis",
            vmin = 0,
            vmax = 100,
            alpha=0.5,
            s=0.8,
            figsize=(10, 10),
        )

        ##DLC Orientation
        dlc_orientation_params_name = "paws_default"
        fields = list(sgp.DLCOrientationSelection.fetch().dtype.fields.keys())
        orient_key = {
            key: val for key, val in cohort_key.items() if key in fields
        }
        orient_key["dlc_orientation_params_name"] = dlc_orientation_params_name
        sgp.DLCOrientationSelection().insert1(orient_key, skip_duplicates=True)
        sgp.DLCOrientation().populate(orient_key)

        ##DLCPosV1
        fields = list(sgp.DLCPosV1.fetch().dtype.fields.keys())
        dlc_key = {
            key: val for key, val in centroid_key.items() if key in fields
        }
        dlc_key["dlc_si_cohort_centroid"] = centroid_key[
            "dlc_si_cohort_selection_name"
        ]
        dlc_key["dlc_si_cohort_orientation"] = orient_key[
            "dlc_si_cohort_selection_name"
        ]
        dlc_key["dlc_orientation_params_name"] = orient_key[
            "dlc_orientation_params_name"
        ]
        sgp.DLCPosSelection().insert1(dlc_key, skip_duplicates=True)
        sgp.DLCPosV1().populate(dlc_key)

    else:
        continue

[20:06:46][INFO] Spyglass: inserted entry into Pose Estimation Selection


{'nwb_file_name': 'wilbur20210512_.nwb', 'epoch': 4, 'video_file_num': 1, 'project_name': 'WtrackSep5-trainset95shuffle1', 'dlc_model_name': 'WtrackSep5-trainset95shuffle1model', 'dlc_model_params_name': 'WtrackSep5', 'dlc_si_cohort_selection_name': 'four_paw_centroid', 'dlc_centroid_params_name': 'four_paw_centroid'}


[20:07:33][INFO] Spyglass: inserted entry into Pose Estimation Selection


{'nwb_file_name': 'wilbur20210512_.nwb', 'epoch': 6, 'video_file_num': 1, 'project_name': 'WtrackSep5-trainset95shuffle1', 'dlc_model_name': 'WtrackSep5-trainset95shuffle1model', 'dlc_model_params_name': 'WtrackSep5', 'dlc_si_cohort_selection_name': 'four_paw_centroid', 'dlc_centroid_params_name': 'four_paw_centroid'}


[20:08:17][INFO] Spyglass: inserted entry into Pose Estimation Selection


{'nwb_file_name': 'wilbur20210512_.nwb', 'epoch': 8, 'video_file_num': 1, 'project_name': 'WtrackSep5-trainset95shuffle1', 'dlc_model_name': 'WtrackSep5-trainset95shuffle1model', 'dlc_model_params_name': 'WtrackSep5', 'dlc_si_cohort_selection_name': 'four_paw_centroid', 'dlc_centroid_params_name': 'four_paw_centroid'}


In [31]:
(sgp.DLCSmoothInterpCohort() & session_restrict & {"dlc_si_cohort_selection_name": "four_paw_centroid"})


dlc_si_cohort_selection_name,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),video_file_num,project_name name of DLC project,dlc_model_name User-friendly model name,dlc_model_params_name
four_paw_centroid,wilbur20210512_.nwb,2,1,WtrackSep5-trainset95shuffle1,WtrackSep5-trainset95shuffle1model,WtrackSep5
four_paw_centroid,wilbur20210512_.nwb,4,1,WtrackSep5-trainset95shuffle1,WtrackSep5-trainset95shuffle1model,WtrackSep5
four_paw_centroid,wilbur20210512_.nwb,6,1,WtrackSep5-trainset95shuffle1,WtrackSep5-trainset95shuffle1model,WtrackSep5
four_paw_centroid,wilbur20210512_.nwb,8,1,WtrackSep5-trainset95shuffle1,WtrackSep5-trainset95shuffle1model,WtrackSep5


In [32]:
e2_pose = (sgp.DLCPoseEstimation() & session_restrict & {"epoch": 2}).fetch_dataframe()
# e2_pose[e2_pose["likelihood"]>0.95].dropna()

In [33]:
e2_pose["forepawL"].plot.scatter(x = "x", y = "y")

<Axes: xlabel='x', ylabel='y'>

In [34]:
si_key = (sgp.DLCSmoothInterpSelection() & session_restrict & {"epoch": 2, "bodypart": "forepawL"}).fetch("KEY", as_dict = True)[0]

In [35]:
(
            sgp.DLCSmoothInterp() & {**si_key}
        ).fetch1_dataframe().plot.scatter(x = "x", y = "y")

<Axes: xlabel='x', ylabel='y'>

In [36]:
(sgp.DLCCentroidParams() & {"dlc_centroid_params_name": "four_paw_centroid"}).fetch()

array([('four_paw_centroid', {'centroid_method': 'four_paw_centroid', 'points': {'paw_FL': 'forepawL', 'paw_FR': 'forepawR', 'paw_HL': 'hindpawL', 'paw_HR': 'hindpawR'}, 'interpolate': True, 'interp_params': {'max_cm_to_interp': 15}, 'smooth': True, 'smoothing_params': {'smoothing_duration': 0.05, 'smooth_method': 'moving_avg'}, 'max_LED_separation': 30, 'speed_smoothing_std_dev': 0.1})],
      dtype=[('dlc_centroid_params_name', 'O'), ('params', 'O')])

In [37]:
(sgp.DLCCentroid() & session_restrict & {"epoch": 2}).fetch1_dataframe().plot.scatter(x = "position_x", y = "position_y")

<Axes: xlabel='position_x', ylabel='position_y'>

In [38]:
(sgp.DLCOrientation() & session_restrict & {"dlc_si_cohort_selection_name":"four_paw_centroid"}).fetch1_dataframe().plot(y = "orientation")

<Axes: xlabel='time'>